Instalar jdk

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

instalar hadoop

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

extrair arquivo

In [3]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

permite interagir com o sistema operacional

In [4]:
import os

criando variavel de ambiente java

In [5]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

criando variavel de ambiente spark

In [6]:
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

pacote Python que facilita a integração do Apache Spark

In [7]:
!pip install -q findspark

importando no projeto

In [8]:
import findspark

iniciando findSpark

In [9]:
findspark.init()

classe central no Spark que fornece uma interface para programação em Spark com DataFrames e Datasets.

In [10]:
from pyspark.sql import SparkSession

In [37]:
from pyspark.sql.functions import col

In [11]:
from pyspark.sql.types import *

criando variavel do sparkSession

In [12]:
sc = SparkSession.builder.master('local[*]').getOrCreate()

In [13]:
spark = SparkSession.builder.getOrCreate()

testando a variavél para ver se esta tudo certo

In [14]:
sc

setando os arquivos vindos da SEFAZ

In [18]:
file_paths = ["/sefaz/Receita_SE_2015.csv", 
              "/sefaz/Receita_SE_2016.csv", 
              "/sefaz/Receita_SE_2017.csv", 
              "/sefaz/Receita_SE_2018.csv", 
              "/sefaz/Receita_SE_2019.csv", 
              "/sefaz/Receita_SE_2020.csv", 
              "/sefaz/Receita_SE_2021.csv", 
              "/sefaz/Receita_SE_2022.csv"]

convertendo dados para dataFrame

In [19]:
dataFrame = spark.read.csv(file_paths)

vefificando se realmente foi convertido a data frame

In [20]:
type(dataFrame)

pyspark.sql.dataframe.DataFrame

verificando o se foi convertido corretamente os tipos dad variaveis

In [21]:
dataFrame.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)



In [26]:
dataFrame = dataFrame.withColumnRenamed("_c1", "SGL_ORGAO") \
       .withColumnRenamed("_c2", "MES") \
       .withColumnRenamed("_c3", "ANO") \
       .withColumnRenamed("_c4", "COD_UNIDADE_GOV") \
       .withColumnRenamed("_c5", "SGL_UNIDADE_GOV") \
       .withColumnRenamed("_c6", "COD_FONTE_RECURSO") \
       .withColumnRenamed("_c7", "NOM_FONTE_RECURSO") \
       .withColumnRenamed("_c8", "COD_NATUREZA") \
       .withColumnRenamed("_c9", "VL_ARRECADADA")

In [29]:
dataFrame = dataFrame.subtract(dataFrame.limit(1))

In [35]:
dataFrame.show()

+---------+---------+---+----+---------------+---------------+-----------------+--------------------+------------+-------------+
|COD_ORGAO|SGL_ORGAO|MES| ANO|COD_UNIDADE_GOV|SGL_UNIDADE_GOV|COD_FONTE_RECURSO|   NOM_FONTE_RECURSO|COD_NATUREZA|VL_ARRECADADA|
+---------+---------+---+----+---------------+---------------+-----------------+--------------------+------------+-------------+
|    24000|    SEASC|  1|2016|         241131|SEASC          |        130005044|Contrapartida do ...|    13250199|        11.76|
|    26000|  SEDURBI|  1|2016|         261061|SEDURBI        |        290000392|EXECUÇÃO DE OBRAS...|    13260000|       258.44|
|    26000|  SEDURBI|  1|2016|         261061|SEDURBI        |        224000279|Convênio SISTEMA ...|    24710500|   1196872.27|
|    32000|    SEMAC|  1|2016|         321031|SEMARH         |        120000352|Contrapartida do ...|    13250199|       180.03|
|    17301|  EMDAGRO|  2|2016|         173011|EMDAGRO        |        270000000|Recursos Diretame

In [38]:
df_filtrado = dataFrame.filter(col("COD_ORGAO") == "16000")

In [ ]:
df_filtrado.show(200)

In [42]:
df_filtrado = df_filtrado.na.drop()

In [43]:
df_filtrado.coalesce(1).write.csv("/sefaz/todos_dados_sefaz-2015-2022", mode="overwrite")